In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from datetime import datetime
import datetime
from collections import Counter
import math

pd.set_option('display.max_columns', None)

In [2]:
game_logistics_df = pd.read_csv('total_game_logistics.csv')
away_stats_df = pd.read_csv('total_away_stats.csv')
home_stats_df = pd.read_csv('total_home_stats.csv')
box_score_df = pd.read_csv('total_box_score_df.csv')
hof_df = pd.read_csv('MLB_hof.csv')
mvp_df = pd.read_csv('MLB_mvp.csv')
allstar_df = pd.read_csv('MLB_allstars.csv')
pitching_line_df = pd.read_csv('pitching_line_df.csv')
cya_df = pd.read_csv('MLB_cya.csv')
team_codes_df = pd.read_csv('team_codes.csv')
games_attended_df = pd.read_csv('sports_attendance.csv')
clemente_award_df = pd.read_csv('clemente_award.csv')
all_MLB_df = pd.read_csv('all_MLB_df.csv')
all_silver_sluggers_df = pd.read_csv('MLB_silver_sluggers.csv')
ERA_champs_df = pd.read_csv('MLB_ERA_champs.csv')
RLOY_df = pd.read_csv('MLB_RLOY.csv')
roy_df = pd.read_csv('MLB_ROY.csv')
all_gold_gloves_df = pd.read_csv('MLB_gold_gloves.csv')
batting_champs_df = pd.read_csv('MLB_batting_champs.csv')
total_home_pitching_df = pd.read_csv('total_home_pitching_stats.csv')
total_away_pitching_df = pd.read_csv('total_away_pitching_stats.csv')
MLB_players_df = pd.read_csv('all_MLB_players.csv')

In [3]:
all_player_stats_df = away_stats_df.append(home_stats_df)

all_player_stats_df.rename(columns = {'Unnamed: 0':'Name'}, inplace = True)

all_player_stats_df = all_player_stats_df[all_player_stats_df.Name != 'Team Totals']

all_player_stats_df = all_player_stats_df.fillna(0)
all_player_stats_df['Games'] = 1
# all_player_stats_df.sort_values(by=['OPS'], ascending=False).head(25)
all_player_stats_df = all_player_stats_df.reset_index()
all_player_stats_df = all_player_stats_df.drop(columns = 'index')

In [4]:
all_player_stats_df.iloc[950:1000]

,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,SLG,OPS,Pit,Str,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Games
950,['Guillermo RodriguezPH'],San Francisco Giants,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.258,0.368,0.290,0.659,6.0,5.0,-0.050,1.91,0.000,-0.050,-0.00%,0.10,-0.2,0.0,0.0,0,7/28/2007,1
951,['Vinnie ChulkP'],San Francisco Giants,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.00,0.000,0,0,0.00,0.0,0.0,0.0,0,7/28/2007,1
952,['Jack TaschnerP'],San Francisco Giants,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.00,0.000,0,0,0.00,0.0,0.0,0.0,0,7/28/2007,1
953,['Mark SweeneyPH'],San Francisco Giants,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.228,0.344,0.405,0.749,8.0,5.0,0.470,5.40,0.470,0.000,0.01%,0.27,1.2,0.0,0.0,2B,7/28/2007,1
954,['Fred LewisPR'],San Francisco Giants,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.274,0.355,0.400,0.755,0.0,0.0,0.000,0.00,0.000,0.000,0%,0.00,0.0,0.0,0.0,0,7/28/2007,1
955,['Skip SchumakerCF'],St. Louis Cardinals,5.0,0.0,1.0,0.0,0.0,0.0,5.0,0.305,0.363,0.417,0.780,14.0,12.0,-0.056,0.73,0.018,-0.074,-0.03%,0.55,-0.7,1.0,0.0,0,9/5/2008,1
956,['Troy Glaus3B'],St. Louis Cardinals,5.0,0.0,1.0,0.0,0.0,1.0,5.0,0.271,0.372,0.476,0.848,14.0,10.0,-0.036,0.80,0.042,-0.079,-0.02%,0.60,-0.4,0.0,2.0,0,9/5/2008,1
957,['Albert Pujols1B'],St. Louis Cardinals,5.0,1.0,2.0,0.0,0.0,0.0,5.0,0.360,0.468,0.642,1.110,16.0,9.0,0.029,1.22,0.131,-0.102,0.01%,0.92,0.1,15.0,1.0,2·2B,9/5/2008,1
958,['Rick AnkielLF'],St. Louis Cardinals,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.264,0.336,0.510,0.846,10.0,7.0,-0.212,1.56,0.000,-0.212,-0.10%,1.18,-1.4,3.0,0.0,GDP,9/5/2008,1
959,['Ryan LudwickRF'],St. Louis Cardinals,4.0,0.0,0.0,0.0,0.0,1.0,4.0,0.298,0.374,0.589,0.963,22.0,16.0,-0.149,1.40,0.000,-0.149,-0.07%,1.05,-0.9,1.0,0.0,0,9/5/2008,1


In [5]:
batting_details_list = all_player_stats_df.Details.tolist()

batting_details_separated_list = []

for i in range(len(batting_details_list)):
    if batting_details_list[i] == 0:
        batting_details_separated_list.append('')
    else:
        x = batting_details_list[i].split(',')
        batting_details_separated_list.append(x)

unique_details_list = []

for i in range(len(batting_details_separated_list)):
    if len(batting_details_separated_list[i]) > 0:
        for j in range(len(batting_details_separated_list[i])):
            if '·' in batting_details_separated_list[i][j]:
                y = batting_details_separated_list[i][j].split('·')
                
                if y[1] not in unique_details_list:
                    unique_details_list.append(batting_details_separated_list[i][j])
            elif batting_details_separated_list[i][j] not in unique_details_list:
                unique_details_list.append(batting_details_separated_list[i][j])
            else:
                continue
               

In [6]:
batting_details_complete_list = []

for i in range(len(batting_details_separated_list)):
    batting_details_temp_list = []

    if len(batting_details_separated_list[i]) == 0:
        for m in range(len(unique_details_list)):
            batting_details_temp_list.append(0)
    elif len(batting_details_separated_list[i]) == 1:
        for n in range(len(unique_details_list)):
            if unique_details_list[n] in batting_details_separated_list[i][0]:
                if '·' in batting_details_separated_list[i][0]:
                    y = batting_details_separated_list[i][0].split('·')
                    batting_details_temp_list.append(int(y[0]))
                else:
                    batting_details_temp_list.append(1)

            else:
                batting_details_temp_list.append(0)      
    else:
        new_temp_list = []
        new_temp_duplicate_list = []
        
        for j in range(len(batting_details_separated_list[i])):
            if '·' in batting_details_separated_list[i][j]:
                y = batting_details_separated_list[i][j].split('·')
                new_temp_list.append(y[1])
                duplicate_int = int(y[0])
                new_temp_duplicate_list.append(duplicate_int)
            else:
                new_temp_list.append(batting_details_separated_list[i][j])
                new_temp_duplicate_list.append(1)
        
        for k in range(len(unique_details_list)):
            if unique_details_list[k] in new_temp_list:
                index_holder = new_temp_list.index(unique_details_list[k])
                batting_details_temp_list.append(new_temp_duplicate_list[index_holder])
                
            else:
                batting_details_temp_list.append(0)

    batting_details_complete_list.append(batting_details_temp_list)
                    
                    
            

In [7]:
batting_details_df = pd.DataFrame(batting_details_complete_list, columns = unique_details_list)
new_all_player_stats_df = pd.merge(all_player_stats_df, batting_details_df, right_index=True, left_index=True)

temp_name_list = new_all_player_stats_df.Name.to_list()

cleaned_name_list = []

for item in temp_name_list:
    temp_name = re.findall(r"\'(.*?)'", item)
    
    if temp_name == []:
        temp_name = re.findall(r'"(.*?)"', item)
    
    cleaned_name_list.append(temp_name[0])

name_pos_list = []

for name in cleaned_name_list:
    for i in range(len(name)):
        j = i + 1

        if name[(-j)] == 'I':
            break
        elif (name[(-j)].isupper() == True) or (name[(-j)] == '-') or (name[(-j)].isnumeric() == True):
            continue
        else:     
            name = name[:(-i)] + '|' + name[(-i):]
            break    
    
    name_pos = name.split('|')
    name_pos_list.append(name_pos)
        
final_name_list = []
pos_list = []
        
for item in name_pos_list:
    final_name_list.append(item[0])
    try:
        pos_list.append(item[1])
    except IndexError:
        pos_list.append('')
        
        
new_all_player_stats_df['Name'] = final_name_list       
new_all_player_stats_df['Pos'] = pos_list

new_all_player_stats_df.sort_values(by=['RBI'], ascending=False).head(50)


,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,SLG,OPS,Pit,Str,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Games,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Pos
1775,Wilmer Flores,San Francisco Giants,3.0,1.0,1.0,6.0,1.0,0.0,5.0,0.269,0.340,0.419,0.759,21.0,12.0,0.245,0.84,0.259,-0.014%,0.15%,0.84,3.2,0.0,2.0,"HR,SF",5/7/2022,1,1,0,1,0,0,0,0,0,0,0,3B
385,Stephen Piscotty,St. Louis Cardinals,4.0,2.0,3.0,5.0,1.0,0.0,5.0,0.324,0.354,0.562,0.916,19.0,9.0,0.130,0.55,0.151,-0.021%,0.07%,0.46,4.8,2.0,0.0,"2·HR,3B,IW",8/23/2015,1,2,0,0,1,0,0,0,0,1,0,LF-RF
1759,Yordan Alvarez,Houston Astros,3.0,3.0,3.0,5.0,1.0,0.0,4.0,0.308,0.378,0.559,0.937,9.0,5.0,0.460,1.10,0.460,0.000%,0.48%,1.92,4.4,1.0,0.0,"2·HR,IW",7/6/2021,1,2,0,0,1,0,0,0,0,0,0,LF
1153,Adrian Gonzalez,Los Angeles Dodgers,4.0,2.0,2.0,4.0,0.0,0.0,4.0,0.293,0.339,0.465,0.804,11.0,10.0,0.079,0.85,0.128,-0.049%,0.01%,0.26,2.2,7.0,1.0,2·HR,8/30/2013,1,2,0,0,0,0,0,0,0,0,0,1B
1494,Ian Happ,Chicago Cubs,5.0,2.0,2.0,4.0,0.0,2.0,5.0,0.235,0.325,0.515,0.839,16.0,12.0,0.365,1.15,0.439,-0.075%,0.30%,1.57,2.9,3.0,0.0,2·HR,6/4/2017,1,2,0,0,0,0,0,0,0,0,0,CF
1455,Victor Martinez,Detroit Tigers,4.0,2.0,2.0,4.0,1.0,0.0,5.0,0.293,0.355,0.483,0.838,14.0,6.0,0.184,0.94,0.210,-0.026%,0.28%,2.37,4.0,0.0,0.0,HR,9/25/2016,1,1,0,0,0,0,0,0,0,0,0,DH
1735,Gerardo Parra,Washington Nationals,3.0,1.0,3.0,4.0,0.0,0.0,4.0,0.224,0.286,0.369,0.655,11.0,7.0,0.252,1.63,0.252,0.000%,0.03%,0.32,3.1,1.0,0.0,"HR,2B,SF",9/27/2019,1,1,1,1,0,0,0,0,0,0,0,CF
1345,Kirk Nieuwenhuis,Milwaukee Brewers,3.0,2.0,3.0,4.0,1.0,0.0,4.0,0.207,0.321,0.380,0.702,17.0,8.0,0.151,0.42,0.159,-0.008%,0.00%,0.01,3.9,2.0,0.0,"2·HR,CS",7/23/2016,1,2,0,0,0,0,0,0,0,0,1,CF
758,Khris Davis,Oakland Athletics,4.0,1.0,1.0,4.0,0.0,1.0,4.0,0.249,0.327,0.544,0.871,11.0,7.0,0.001,0.80,0.055,-0.054%,0%,0.78,2.1,0.0,0.0,HR,9/16/2018,1,1,0,0,0,0,0,0,0,0,0,DH
1597,Jose Martinez,St. Louis Cardinals,4.0,1.0,2.0,4.0,0.0,0.0,4.0,0.357,0.424,0.571,0.996,12.0,8.0,0.315,1.11,0.323,-0.007%,0.17%,1.02,2.9,7.0,1.0,HR,4/7/2018,1,1,0,0,0,0,0,0,0,0,0,1B


In [8]:
col_list = new_all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('Team')
col_list.remove('BA')
col_list.remove('OBP')
col_list.remove('SLG')
col_list.remove('OPS')
col_list.remove('Pit')
col_list.remove('Str')
col_list.remove('aLI')
col_list.remove('WPA+')
col_list.remove('WPA-')
col_list.remove('cWPA')
col_list.remove('acLI')
col_list.remove('RE24')
col_list.remove('Details')
col_list.remove('Date')

total_sum_list = []

for col in col_list: 
    sum_list = []
    if col == "Team" or col == 'Pos':           
        players_list = new_all_player_stats_df["Name"].values.tolist()
        player_pos_list = new_all_player_stats_df["Pos"].values.tolist()        
        teams_list = new_all_player_stats_df["Team"].values.tolist()
        new_teams_list = []
        new_pos_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
        for i in range(len(player_pos_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(player_pos_list[i])
            new_pos_list.append(temp_list)
        
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()
        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')
        
        new_pos_df = pd.DataFrame(new_pos_list, columns = ("Name", "Pos"))
        new_pos_df = new_pos_df.drop_duplicates()
        new_pos_df = new_pos_df.groupby('Name')['Pos'].apply('/'.join).reset_index()
        new_pos_df = new_pos_df.set_index('Name')
        
    
    else:
        sum_list = new_all_player_stats_df.groupby('Name')[col].sum()
        total_sum_list.append(sum_list)
    
    
    
player_stat_sum_df = pd.DataFrame(total_sum_list)
player_stat_sum_df = player_stat_sum_df.transpose()


H_list = player_stat_sum_df['H'].tolist()
AB_list = player_stat_sum_df['AB'].tolist()
BB_list = player_stat_sum_df['BB'].tolist()
IW_list = player_stat_sum_df['IW'].tolist()
HBP_list = player_stat_sum_df['HBP'].tolist()
SF_list = player_stat_sum_df['SF'].tolist()
double_list = player_stat_sum_df['2B'].tolist()
triple_list = player_stat_sum_df['3B'].tolist()
HR_list = player_stat_sum_df['HR'].tolist()

BA_list = []
for i in range(len(AB_list)):
    if AB_list[i] == 0:
        BA_list.append(.000)
    else:
        BA = H_list[i] / AB_list[i]
        BA_list.append(BA)

player_stat_sum_df['BA'] = BA_list
player_stat_sum_df.loc[:, "BA"] = player_stat_sum_df["BA"].map('{:.3f}'.format)


# OBP Forumla = (H + BB + HBP) / (AB + BB + HBP + SF)

OBP_list = []
for i in range(len(AB_list)):
    if float(AB_list[i] + BB_list[i] + HBP_list[i] + SF_list[i]) == 0:
        OBP_list.append(.000)
    else:
        OBP = ((H_list[i] + BB_list[i] + HBP_list[i]) / (AB_list[i] + BB_list[i] + HBP_list[i] + SF_list[i]))
        OBP_list.append(OBP)

player_stat_sum_df['OBP'] = OBP_list
player_stat_sum_df.loc[:, "OBP"] = player_stat_sum_df["OBP"].map('{:.3f}'.format)


# (1B + 2Bx2 + 3Bx3 + HRx4)/AB

SLG_list = []
for i in range(len(AB_list)):
    if AB_list[i] == 0:
        SLG_list.append(.000)
    else:
        SLG = (((H_list[i] - double_list[i] - triple_list[i] - HR_list[i]) + (2*double_list[i]) + (3*triple_list[i]) + (4*HR_list[i]))/ (AB_list[i]))
        SLG_list.append(SLG)

player_stat_sum_df['SLG'] = SLG_list
player_stat_sum_df.loc[:, "SLG"] = player_stat_sum_df["SLG"].map('{:.3f}'.format)


OPS_list = []
for i in range(len(OBP_list)):
    OPS = OBP_list[i] + SLG_list[i]
    OPS_list.append(OPS)

player_stat_sum_df['OPS'] = OPS_list
player_stat_sum_df.loc[:, "OPS"] = player_stat_sum_df["OPS"].map('{:.3f}'.format)


player_stat_sum_df = player_stat_sum_df.join(new_teams_df)
player_stat_sum_df = player_stat_sum_df.join(new_pos_df)

player_stat_sum_df.sort_values('Games', ascending = False).head(50)


,AB,R,H,RBI,BB,SO,PA,WPA,PO,A,Games,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,BA,OBP,SLG,OPS,Team,Pos
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,
,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,Minnesota Twins/Toronto Blue Jays/Cincinnati R...,
Yadier Molina,67.0,5.0,17.0,9.0,4.0,11.0,75.0,0.161,152.0,6.0,19.0,1.0,0.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.254,0.288,0.299,0.586,St. Louis Cardinals,C-1B/C
Matt Carpenter,52.0,5.0,9.0,4.0,3.0,20.0,58.0,-0.212,44.0,19.0,15.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.173,0.224,0.231,0.455,St. Louis Cardinals/San Diego Padres,PH/2B/3B/1B
Tommy Pham,31.0,4.0,2.0,0.0,8.0,12.0,39.0,-0.241,16.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065,0.256,0.065,0.321,St. Louis Cardinals,CF-LF/CF/PH/LF
Jedd Gyorko,34.0,4.0,8.0,1.0,3.0,7.0,37.0,-0.386,7.0,20.0,10.0,1.0,3.0,0.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.235,0.297,0.412,0.709,San Diego Padres/St. Louis Cardinals,2B/SS/3B
David Freese,23.0,4.0,6.0,7.0,5.0,7.0,28.0,0.463,7.0,8.0,9.0,4.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.261,0.393,0.870,1.262,St. Louis Cardinals/Pittsburgh Pirates/Los Ang...,3B/1B/DH/PH
Adrian Gonzalez,29.0,4.0,7.0,8.0,1.0,6.0,31.0,-0.117,76.0,6.0,9.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.241,0.258,0.448,0.706,Los Angeles Dodgers,1B/PH-1B
Stephen Piscotty,32.0,6.0,15.0,9.0,4.0,3.0,36.0,0.367,16.0,0.0,9.0,3.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,2.0,1.0,0.469,0.528,0.938,1.465,St. Louis Cardinals/Oakland Athletics,LF-RF/LF/RF/CF
Greg Garcia,11.0,0.0,1.0,0.0,3.0,2.0,14.0,-0.126,1.0,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.091,0.286,0.091,0.377,St. Louis Cardinals,2B/PH/3B/PH-2B


In [9]:
col_list = new_all_player_stats_df.columns.tolist()
col_list.remove('Name')
col_list.remove('BA')
col_list.remove('SLG')
col_list.remove('OPS')
col_list.remove('Pos')
col_list.remove('OBP')
col_list.remove('Pit')
col_list.remove('Str')
col_list.remove('WPA')
col_list.remove('aLI')
col_list.remove('WPA+')
col_list.remove('WPA-')
col_list.remove('cWPA')
col_list.remove('Details')
col_list.remove('Date')
col_list.remove('acLI')
col_list.remove('RE24')

total_average_list = []

for col in col_list: 
    average_list = []
    teams_list = []
    
    if col == "Games":
        average_list = new_all_player_stats_df.groupby('Name')[col].sum()
        total_average_list.append(average_list)
        
    elif col == "Team":           
        players_list = new_all_player_stats_df["Name"].values.tolist()
        teams_list = new_all_player_stats_df["Team"].values.tolist()
        new_teams_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()

        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')

    else:
        average_list = new_all_player_stats_df.groupby('Name')[col].mean()
        total_average_list.append(average_list)
    
temp_player_stat_average_df = pd.DataFrame(total_average_list)
temp_player_stat_average_df = temp_player_stat_average_df.transpose()
player_stat_average_df = pd.concat([temp_player_stat_average_df, new_teams_df], axis=1, join="inner")

player_stat_average_df = player_stat_average_df.round(2)

first_column2 = player_stat_average_df.pop('Games')
second_column2 = player_stat_average_df.pop('Team')

player_stat_average_df.insert(0, 'Team', second_column2)
player_stat_average_df.insert(0, 'Games', first_column2)

player_stat_average_df.sort_values(by=['Games'], ascending=False).head(40)

,Games,Team,AB,R,H,RBI,BB,SO,PA,PO,A,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS
Name,,,,,,,,,,,,,,,,,,,,,
,44.0,Minnesota Twins/Toronto Blue Jays/Cincinnati R...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Yadier Molina,19.0,St. Louis Cardinals,3.53,0.26,0.89,0.47,0.21,0.58,3.95,8.00,0.32,0.05,0.00,0.11,0.00,0.00,0.11,0.05,0.00,0.00,0.05
Matt Carpenter,15.0,St. Louis Cardinals/San Diego Padres,3.47,0.33,0.60,0.27,0.20,1.33,3.87,2.93,1.27,0.00,0.20,0.13,0.00,0.00,0.00,0.00,0.07,0.00,0.00
Tommy Pham,11.0,St. Louis Cardinals,2.82,0.36,0.18,0.00,0.73,1.09,3.55,1.45,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Jedd Gyorko,10.0,San Diego Padres/St. Louis Cardinals,3.40,0.40,0.80,0.10,0.30,0.70,3.70,0.70,2.00,0.10,0.30,0.00,0.00,0.40,0.00,0.10,0.00,0.00,0.00
David Freese,9.0,St. Louis Cardinals/Pittsburgh Pirates/Los Ang...,2.56,0.44,0.67,0.78,0.56,0.78,3.11,0.78,0.89,0.44,0.22,0.00,0.11,0.11,0.00,0.00,0.00,0.00,0.00
Adrian Gonzalez,9.0,Los Angeles Dodgers,3.22,0.44,0.78,0.89,0.11,0.67,3.44,8.44,0.67,0.22,0.00,0.11,0.00,0.11,0.00,0.00,0.00,0.00,0.00
Stephen Piscotty,9.0,St. Louis Cardinals/Oakland Athletics,3.56,0.67,1.67,1.00,0.44,0.33,4.00,1.78,0.00,0.33,0.22,0.00,0.11,0.22,0.00,0.00,0.00,0.22,0.11
Greg Garcia,8.0,St. Louis Cardinals,1.38,0.00,0.12,0.00,0.38,0.25,1.75,0.12,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [10]:
box_score_df = box_score_df.tail(-1)
box_score_df = box_score_df.rename(columns={"Unnamed: 0": "Date", "Unnamed: 2": "Details"})
box_score_df = box_score_df.drop(columns=['Home_Away'])

temp_box_score_df = box_score_df.dropna(subset = ['R'])

index_list = temp_box_score_df.index.tolist()
home_away_list = []

for i in range(len(index_list)):
    if (i % 2) == 0:
        home_away_list.append('Away')
    else:
        home_away_list.append('Home')

temp_box_score_df.insert(1, 'Home_Away', home_away_list)


temp_box_score_df.tail(60)

,Date,Home_Away,Details,1,10,11,12,13,14,15,2,3,4,5,6,7,8,9,E,H,R,Team
102,"('7/26/2016',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.0,7.0,2.0,Atlanta Braves
103,"('7/26/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,5.0,0.0,Minnesota Twins
105,"('7/29/2016',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,8.0,1.0,Seattle Mariners
106,"('7/29/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,0.0,0.0,6.0,1.0,0.0,X,0.0,14.0,12.0,Chicago Cubs
108,"('8/7/2016',)",Away,via Sports Logos.netAbout logos,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,11.0,6.0,Atlanta Braves
109,"('8/7/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0,0.0,9.0,3.0,St. Louis Cardinals
111,"('9/3/2016',)",Away,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,5.0,1.0,St. Louis Cardinals
112,"('9/3/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,2.0,5.0,X,1.0,12.0,9.0,Cincinnati Reds
114,"('9/20/2016',)",Away,via Sports Logos.netAbout logos,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0,2.0,13.0,6.0,Pittsburgh Pirates
115,"('9/20/2016',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0,0.0,7.0,3.0,Milwaukee Brewers


In [11]:
date_list = temp_box_score_df['Date'].tolist()
final_score_list = temp_box_score_df['R'].tolist()
win_loss_list = []
home_team_wins = 0
away_team_wins = 0
home_team_losses = 0
away_team_losses = 0
j = 1

try:
    for i in range(len(final_score_list)):

        if (i + 2) != len(final_score_list) and date_list[i] == date_list[i+2]:
            date_list[i] = str(date_list[i]) + 'Game 1'
            date_list[i + 1] = str(date_list[i + 1]) + 'Game 1'
            date_list[i + 2] = str(date_list[i + 2]) + 'Game 2'
            date_list[i + 3] = str(date_list[i + 3]) + 'Game 2'
        if date_list[i] == date_list[i+1]:
            print(f'Game {j}: {date_list[i]}')
            if final_score_list[i] > final_score_list[i + 1]:
                win_loss_list.append('W')
                win_loss_list.append('L')
                away_team_wins = away_team_wins + 1
                home_team_losses = home_team_losses + 1
                print('Away Team Won')
                j = j + 1
            else:
                win_loss_list.append('L')
                win_loss_list.append('W')
                home_team_wins = home_team_wins + 1
                away_team_losses = away_team_losses + 1
                print('Home Team Won')
                j = j + 1
        else:
            continue
except IndexError:
    print("An exception was made")

home_win_loss_percentage = (home_team_wins/(home_team_wins + home_team_losses))
away_win_loss_percentage = (away_team_wins/(away_team_wins + away_team_losses))

Game 1: ('7/27/2007',)
Home Team Won
Game 2: ('7/28/2007',)
Home Team Won
Game 3: ('9/5/2008',)
Away Team Won
Game 4: ('9/6/2008',)
Home Team Won
Game 5: ('7/25/2009',)
Home Team Won
Game 6: ('7/26/2009',)
Away Team Won
Game 7: ('8/22/2009',)
Away Team Won
Game 8: ('4/24/2010',)
Home Team Won
Game 9: ('9/26/2010',)
Home Team Won
Game 10: ('10/28/2010',)
Home Team Won
Game 11: ('7/14/2012',)
Home Team Won
Game 12: ('9/11/2012',)
Home Team Won
Game 13: ('10/14/2012',)
Away Team Won
Game 14: ('5/24/2013',)
Away Team Won
Game 15: ('8/21/2013',)
Away Team Won
Game 16: ('8/30/2013',)
Home Team Won
Game 17: ('6/26/2014',)
Home Team Won
Game 18: ('7/5/2014',)
Home Team Won
Game 19: ('7/11/2014',)
Home Team Won
Game 20: ('7/20/2014',)
Home Team Won
Game 21: ('8/7/2014',)
Away Team Won
Game 22: ('8/31/2014',)
Home Team Won
Game 23: ('10/2/2014',)
Away Team Won
Game 24: ('5/23/2015',)
Home Team Won
Game 25: ('6/6/2015',)
Home Team Won
Game 26: ('6/19/2015',)
Away Team Won
Game 27: ('8/23/2015',)


In [12]:
i

123

In [13]:
home_team_record = ['Home',home_team_wins,home_team_losses,home_win_loss_percentage]
away_team_record = ['Away',away_team_wins,away_team_losses,away_win_loss_percentage]

home_away_record_df = pd.DataFrame(columns = ['', 'Wins', 'Losses','Win%'])
  
home_away_record_df.loc[len(home_away_record_df)] = home_team_record
home_away_record_df.loc[len(home_away_record_df)] = away_team_record
home_away_record_df = home_away_record_df.set_index('')
home_away_record_df.loc[:, "Win%"] = home_away_record_df["Win%"].map('{:.3f}'.format)

home_away_record_df

,Wins,Losses,Win%
,,,
Home,39,23,0.629
Away,23,39,0.371


In [14]:
temp_box_score_df['Decision'] = win_loss_list
temp_box_score_df

/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/2583124398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_box_score_df['Decision'] = win_loss_list


,Date,Home_Away,Details,1,10,11,12,13,14,15,2,3,4,5,6,7,8,9,E,H,R,Team,Decision
1,"('7/27/2007',)",Away,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,4.0,1.0,0.0,1.0,1.0,0.0,2,2.0,15.0,10.0,Florida Marlins,L
2,"('7/27/2007',)",Home,via Sports Logos.netAbout logos,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,4.0,6.0,0.0,0.0,X,0.0,14.0,12.0,San Francisco Giants,W
4,"('7/28/2007',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0,2.0,6.0,3.0,Florida Marlins,L
5,"('7/28/2007',)",Home,via Sports Logos.netAbout logos,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,0.0,9.0,4.0,San Francisco Giants,W
8,"('9/5/2008',)",Away,via Sports Logos.netAbout logos,0.0,0.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,9.0,4.0,Florida Marlins,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,"('6/17/2023',)",Home,via Sports Logos.netAbout logos,2.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2.0,8.0,3.0,Seattle Mariners,L
189,"('7/15/2023',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0,1.0,6.0,4.0,Boston Red Sox,L
190,"('7/15/2023',)",Home,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,6.0,2.0,0.0,0.0,2.0,0.0,X,0.0,12.0,10.0,Chicago Cubs,W
192,"('9/3/2023',)",Away,via Sports Logos.netAbout logos,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,4.0,0.0,San Francisco Giants,L


In [15]:
team_records_df = pd.DataFrame(columns = ['Team', 'Games', 'Wins', 'Losses','Win%'])

for item in temp_box_score_df.Team.unique():
    team_wins = len(temp_box_score_df.loc[(temp_box_score_df['Team'] == item) & (temp_box_score_df['Decision'] == 'W')].index)
    team_losses = len(temp_box_score_df.loc[(temp_box_score_df['Team'] == item) & (temp_box_score_df['Decision'] == 'L')].index)
    
    team_win_percentage = (team_wins/(team_wins + team_losses))
    team_total_games = (team_wins + team_losses)
    
    team_record_list = [item,team_total_games,team_wins,team_losses,team_win_percentage]
    team_records_df.loc[len(team_records_df)] = team_record_list

team_records_df = team_records_df.set_index('Team')
team_records_df.loc[:, "Win%"] = team_records_df["Win%"].map('{:.3f}'.format)

team_records_df.sort_values(by=['Games','Win%'], ascending=False)

,Games,Wins,Losses,Win%
Team,,,,
St. Louis Cardinals,21,9,12,0.429
San Francisco Giants,11,7,4,0.636
Los Angeles Dodgers,11,5,6,0.455
Chicago Cubs,6,5,1,0.833
San Diego Padres,6,2,4,0.333
Florida Marlins,5,2,3,0.400
Oakland Athletics,5,2,3,0.400
Chicago White Sox,4,3,1,0.750
Los Angeles Angels of Anaheim,4,2,2,0.500


In [16]:
player_records_df = pd.DataFrame(columns = ['Name', 'Games', 'Wins', 'Losses','Win%'])

temp_date_list = temp_box_score_df['Date'].to_list()
cleaned_dates_list = []

temp_box_score_df

for item in temp_date_list:
    cleaned_date = re.findall(r"'([^']*)'", item)
    cleaned_dates_list.append(cleaned_date[0])


temp_box_score_df.drop('Date', axis = 1, inplace = True)
temp_box_score_df['Date'] = cleaned_dates_list

for player in new_all_player_stats_df.Name.unique():
    
    temp_df = new_all_player_stats_df.loc[(new_all_player_stats_df['Name'] == player)]
    temp_merge_df = pd.merge(temp_df, temp_box_score_df, on=['Team','Date'])
         
    player_wins = 0
    player_losses = 0
        
    for item in temp_merge_df['Decision']:
        if item == 'L':
            player_losses = player_losses + 1
        else:
            player_wins = player_wins + 1

        player_win_percentage = (player_wins/(player_wins + player_losses))
        player_total_games = (player_wins + player_losses)

        player_record_list = [player,player_total_games,player_wins,player_losses,player_win_percentage]
        player_records_df.loc[len(player_records_df)] = player_record_list

player_records_df.loc[:, "Win%"] = player_records_df["Win%"].map('{:.3f}'.format)
player_records_df = player_records_df.sort_values(by=['Games'], ascending=False)                                      

player_records_df = player_records_df.drop_duplicates(subset=['Name'])
player_records_df = player_records_df.set_index('Name')

player_records_df = player_records_df.sort_values(by=['Games','Win%'], ascending=False)                                      

player_records_df


/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/2114474786.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_box_score_df['Date'] = cleaned_dates_list


,Games,Wins,Losses,Win%
Name,,,,
,44,22,22,0.500
Yadier Molina,19,9,10,0.474
Matt Carpenter,15,7,8,0.467
Tommy Pham,11,5,6,0.455
Jedd Gyorko,10,2,8,0.200
...,...,...,...,...
David Holmberg,1,0,1,0.000
Brandon McCarthy,1,0,1,0.000
Brandon Morrow,1,0,1,0.000


In [17]:
player_stat_average_df.reset_index(inplace=True)
hof_seen_df = pd.merge(player_stat_average_df, hof_df, on="Name")
hof_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)
hof_seen_df.rename(columns = {'Unnamed: 2': 'Life'}, inplace = True)

hof_seen_df

,Name,Games,Team,AB,R,H,RBI,BB,SO,PA,PO,A,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Life,Voted By,Inducted As,Votes,% of Ballots
0,Billy Hamilton,2.0,Cincinnati Reds,4.0,1.5,1.0,0.0,0.5,1.0,4.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,1866-1940,Veterans,Player,NaN,NaN
1,David Ortiz,1.0,Boston Red Sox,3.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1975-Living,BBWAA,Player,307.0,77.9%
2,Derek Jeter,1.0,New York Yankees,5.0,0.0,1.0,1.0,0.0,1.0,5.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1974-Living,BBWAA,Player,396.0,99.7%
3,Joe Mauer,2.0,Minnesota Twins,4.5,0.5,1.0,0.0,0.0,0.5,4.5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1983-Living,BBWAA,Player,293.0,76.1%
4,Tony Gwynn,2.0,San Diego Padres/Philadelphia Phillies,2.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1960-2014,BBWAA,Player,532.0,97.6%


In [18]:
player_stat_average_df.reset_index(inplace=True)
player_stat_average_df.rename(columns={'Name': 'Player'}, inplace=True)
mvp_seen_df = pd.merge(player_stat_average_df, mvp_df, on="Player")
mvp_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)


mvp_seen_df

,index,Player,Games,Team,AB,R,H,RBI,BB,SO,PA,PO,A,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Year,League
0,7,Aaron Judge,2.0,New York Yankees,4.00,0.00,1.00,0.00,0.00,2.00,4.00,3.50,0.00,0.00,0.50,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2022,AL
1,36,Albert Pujols,7.0,St. Louis Cardinals/Los Angeles Angels of Anaheim,4.00,0.57,0.71,0.43,0.43,0.57,4.43,7.71,0.71,0.14,0.43,0.0,0.00,0.29,0.0,0.0,0.0,0.0,0.0,2009,NL
2,36,Albert Pujols,7.0,St. Louis Cardinals/Los Angeles Angels of Anaheim,4.00,0.57,0.71,0.43,0.43,0.57,4.43,7.71,0.71,0.14,0.43,0.0,0.00,0.29,0.0,0.0,0.0,0.0,0.0,2008,NL
3,36,Albert Pujols,7.0,St. Louis Cardinals/Los Angeles Angels of Anaheim,4.00,0.57,0.71,0.43,0.43,0.57,4.43,7.71,0.71,0.14,0.43,0.0,0.00,0.29,0.0,0.0,0.0,0.0,0.0,2005,NL
4,58,Andrew McCutchen,2.0,Pittsburgh Pirates,4.50,0.00,1.50,1.50,0.00,1.00,4.50,2.00,0.00,0.00,0.50,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2013,NL
5,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,4.50,2.00,0.00,0.50,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2004,NL
6,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,4.50,2.00,0.00,0.50,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2003,NL
7,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,4.50,2.00,0.00,0.50,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2002,NL
8,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,4.50,2.00,0.00,0.50,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2001,NL
9,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,4.50,2.00,0.00,0.50,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1993,NL


In [19]:
allstar_df

,Unnamed: 0,Lineup Spot,Player,Position,Year
0,0,1.0,Pepper Martin,3B,1933: Game 1
1,1,2.0,Frankie Frisch,2B,1933: Game 1
2,2,3.0,Chuck Klein,RF,1933: Game 1
3,3,4.0,Chick Hafey,LF,1933: Game 1
4,4,5.0,Bill Terry,1B,1933: Game 1
...,...,...,...,...,...
6112,6112,NaN,Pablo López,P,2023: Game 1
6113,6113,NaN,Michael Lorenzen,P,2023: Game 1
6114,6114,NaN,Shane McClanahan,P,2023: Game 1
6115,6115,NaN,Jordan Romano,P,2023: Game 1


In [20]:
all_star_list = allstar_df['Player'].to_list()

i = 0
allstars_seen_list = []
allstar_appearance_counter_list = []

for player in new_all_player_stats_df.Name.unique():
    allstar_appearance_counter = 0
    
    for i in range(len(all_star_list)):
        if (player in all_star_list[i]) == True:
            if player not in allstars_seen_list:
                allstars_seen_list.append(player)
                allstar_appearance_counter = allstar_appearance_counter + 1
            else:
                allstar_appearance_counter = allstar_appearance_counter + 1
    
    allstar_appearance_counter_list.append(allstar_appearance_counter)
    i = i + 1            
    
allstars_seen_counter_list = [i for i in allstar_appearance_counter_list if i != 0]
    
allstars_seen_df = pd.DataFrame(columns = ['Player', 'Appearances'])
allstars_seen_df['Player'] = allstars_seen_list
allstars_seen_df['Appearances'] = allstars_seen_counter_list
allstars_seen_df.sort_values(by=['Appearances'], ascending=False)

,Player,Appearances
10,,6117
212,Tony Gwynn,15
194,Barry Bonds,14
239,Derek Jeter,14
1,Miguel Cabrera,12
...,...,...
202,Ryan Franklin,1
113,Jim Johnson,1
204,Chone Figgins,1
112,Erick Aybar,1


In [21]:
# all_gold_gloves_seen_df = pd.merge(player_stat_average_df, all_gold_gloves_df, on="Player")

# all_gold_gloves_seen_df.drop('level_0', axis = 1, inplace = True)
# all_gold_gloves_seen_df.drop('index', axis = 1, inplace = True)
# all_gold_gloves_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)

# # all_gold_gloves_seen_df.sort_values(by=['X_DPOY'], ascending=False)
# all_gold_gloves_seen_df

## 

In [22]:
home_teams_seen_list = games_attended_df['Home'].unique()
away_teams_seen_list = games_attended_df['Visitor'].unique()
all_teams_seen_list = list(home_teams_seen_list)

# for away_team in away_teams_seen_list:
#     all_teams_seen_list.append(away_team)

[all_teams_seen_list.append(x) for x in away_teams_seen_list if x not in all_teams_seen_list]

    
mlb_teams_list = [x for x in team_codes_df['MLB_Teams'].to_list() if str(x) != 'nan']
nba_teams_list = [x for x in team_codes_df['NBA_Teams'].to_list() if str(x) != 'nan']
nfl_teams_list = [x for x in team_codes_df['NFL_Teams'].to_list() if str(x) != 'nan']
nhl_teams_list = [x for x in team_codes_df['NHL_Teams'].to_list() if str(x) != 'nan']
mls_teams_list = [x for x in team_codes_df['MLS_Codes'].to_list() if str(x) != 'nan']

home_mlb_teams_seen_list = []
home_nba_teams_seen_list = []
home_nfl_teams_seen_list = []
home_nhl_teams_seen_list = []
home_mls_teams_seen_list = []

for team in home_teams_seen_list:
    if team in mlb_teams_list:
        home_mlb_teams_seen_list.append(team)
    elif team in nba_teams_list:
        home_nba_teams_seen_list.append(team)
    elif team in nfl_teams_list:
        home_nfl_teams_seen_list.append(team)
    elif team in nhl_teams_list:
        home_nhl_teams_seen_list.append(team)
    else: 
        team = team.replace(' ', '-')
        for mls_team in mls_teams_list:
            if team in mls_team:
                home_mls_teams_seen_list.append(team)
            else:
                continue

                
all_mlb_teams_seen_list = []
all_nba_teams_seen_list = []
all_nfl_teams_seen_list = []
all_nhl_teams_seen_list = []
all_mls_teams_seen_list = []

for team in all_teams_seen_list:
    if team in mlb_teams_list:
        all_mlb_teams_seen_list.append(team)
    elif team in nba_teams_list:
        all_nba_teams_seen_list.append(team)
    elif team in nfl_teams_list:
        all_nfl_teams_seen_list.append(team)
    elif team in nhl_teams_list:
        all_nhl_teams_seen_list.append(team)
    else: 
        team = team.replace(' ', '-')
        for mls_team in mls_teams_list:
            if team in mls_team:
                all_mls_teams_seen_list.append(team)
            else:
                continue
                
mlb_stadiums_visited = (len(home_mlb_teams_seen_list)/((len(mlb_teams_list))-1))
nba_stadiums_visited = (len(home_nba_teams_seen_list)/len(nba_teams_list))
nfl_stadiums_visited = (len(home_nfl_teams_seen_list)/len(nfl_teams_list))
nhl_stadiums_visited = (len(home_nhl_teams_seen_list)/len(nhl_teams_list))
mls_stadiums_visited = (len(home_mls_teams_seen_list)/len(mls_teams_list))

mlb_teams_seen = (len(all_mlb_teams_seen_list)/len(mlb_teams_list))
nba_teams_seen = (len(all_nba_teams_seen_list)/len(nba_teams_list))
nfl_teams_seen = (len(all_nfl_teams_seen_list)/len(nfl_teams_list))
nhl_teams_seen = (len(all_nhl_teams_seen_list)/len(nhl_teams_list))
mls_teams_seen = (len(all_mls_teams_seen_list)/len(mls_teams_list))

games_dict = {}

games_dict['League'] = ['MLB','NBA','NFL','NHL','MLS']
games_dict['League_Teams_Total'] = [((len(mlb_teams_list))-1),len(nba_teams_list),len(nfl_teams_list),len(nhl_teams_list),len(mls_teams_list)]
games_dict['Stadiums_Visited'] = [len(home_mlb_teams_seen_list),len(home_nba_teams_seen_list),len(home_nfl_teams_seen_list),len(home_nhl_teams_seen_list),len(home_mls_teams_seen_list)]
games_dict['%_Visited'] = [mlb_stadiums_visited, nba_stadiums_visited, nfl_stadiums_visited, nhl_stadiums_visited, mls_stadiums_visited]
games_dict['Teams_Seen'] = [len(all_mlb_teams_seen_list),len(all_nba_teams_seen_list),len(all_nfl_teams_seen_list),len(all_nhl_teams_seen_list),len(all_mls_teams_seen_list)]
games_dict['%_Teams_Seen'] = [mlb_teams_seen,nba_teams_seen,nfl_teams_seen,nhl_teams_seen,mls_teams_seen]


games_breakdown_df = pd.DataFrame(games_dict)
games_breakdown_df.loc[:, "%_Visited"] = games_breakdown_df["%_Visited"].map('{:.1%}'.format)
games_breakdown_df.loc[:, "%_Teams_Seen"] = games_breakdown_df["%_Teams_Seen"].map('{:.1%}'.format)
games_breakdown_df.set_index('League')


,League_Teams_Total,Stadiums_Visited,%_Visited,Teams_Seen,%_Teams_Seen
League,,,,,
MLB,32,27,84.4%,32,97.0%
NBA,30,9,30.0%,21,70.0%
NFL,32,1,3.1%,2,6.2%
NHL,32,2,6.2%,5,15.6%
MLS,28,1,3.6%,2,7.1%


In [23]:
# total_home_pitching_df
# total_away_pitching_df

total_pitching_df = pd.concat([total_home_pitching_df, total_away_pitching_df])
total_pitching_df = total_pitching_df.rename(columns={"Unnamed: 0": "Name"})
total_pitching_df


,Name,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str,Ctct,StS,StL,GB,FB,LD,Unk,GSc,IR,IS,WPA,aLI,cWPA,acLI,RE24,Date
0,Barry Zito,San Francisco Giants,4.0,8,6,6,4,3,1,5.28,23,87,50,28,10,12,6,10,4,0,21.0,NaN,NaN,-0.406,1.03,-0.01%,0.04,-3.9,7/27/2007
1,Kevin Correia,San Francisco Giants,1.1,2,1,1,1,2,1,4.21,7,27,15,7,4,4,1,3,1,0,NaN,0.0,0.0,-0.138,0.59,-0.00%,0.02,-0.5,7/27/2007
2,"Vinnie Chulk, W (4-2)",San Francisco Giants,0.2,0,0,0,1,0,0,3.35,3,10,5,3,1,1,0,2,0,0,NaN,1.0,0.0,0.059,1.80,0.00%,0.08,0.6,7/27/2007
3,Jonathan Sánchez,San Francisco Giants,1.2,2,1,1,1,1,0,4.45,8,32,19,13,3,3,2,4,1,0,NaN,0.0,0.0,0.017,0.52,0.00%,0.02,-0.3,7/27/2007
4,Randy Messenger,San Francisco Giants,1.1,3,2,2,0,1,0,2.15,7,22,15,10,2,3,4,2,0,0,NaN,1.0,0.0,0.027,1.04,0.00%,0.05,-1.1,7/27/2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,Brandon Walter,Boston Red Sox,2.0,6,2,2,0,3,0,3.29,11,34,24,14,5,5,3,5,3,0,NaN,0.0,0.0,-0.011,0.08,-0.01%,0.07,-0.9,7/15/2023
324,Boston Red Sox Totals,Boston Red Sox,8.0,12,10,10,4,11,2,11.25,40,150,96,55,21,20,10,14,8,0,30.0,0.0,0.0,-0.326,0.45,-0.17%,0.40,-5.6,7/15/2023
325,"Alex Cobb, L (7-6)",San Francisco Giants,3.0,6,4,4,0,3,2,3.74,15,58,43,24,6,13,8,4,4,0,34.0,NaN,NaN,-0.247,0.72,-0.18%,0.87,-2.5,9/3/2023
326,Keaton Winn,San Francisco Giants,5.0,4,0,0,1,2,0,3.33,19,70,44,27,5,12,10,6,1,0,NaN,0.0,0.0,0.046,0.20,0.03%,0.24,2.5,9/3/2023


In [24]:
# total_pitching_df.filter(like='Totals', axis=0)

pitchers_only_df = total_pitching_df[total_pitching_df['Name'].str.contains("Totals") == False]
pitchers_only_df.columns

Index(['Name', 'Team', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'ERA', 'BF',
       'Pit', 'Str', 'Ctct', 'StS', 'StL', 'GB', 'FB', 'LD', 'Unk', 'GSc',
       'IR', 'IS', 'WPA', 'aLI', 'cWPA', 'acLI', 'RE24', 'Date'],
      dtype='object')

In [25]:
# Game Score = 50 + 3(Outs) + 2(IP after 4) + (SO) - 2(H) - 4(ER) - 2(R - ER) - (BB)

# pitchers_only_df['Game Score'] = 50 + (3 * pitchers_only_df['IP']) + (2 * (pitchers_only_df['IP'] - 4))
# pitchers_only_df


pitchers_only_list = pitchers_only_df.values.tolist()
game_score_list = []

for item in pitchers_only_list:
    
    innings_separated = math.modf(item[2])
    outs_score = (3 * innings_separated[1]) + (round(innings_separated[0],1) * 10)

    if innings_separated[1] > 4:
        innings_after_four_score = (2 * (innings_separated[1] - 4))
    else: 
        innings_after_four_score = 0
        
    strikeouts_score = item[7]
    
    hits_score = (2 * item[3])
    
    earned_runs_score = (4 * item[5])
    
    runs_score = (2 * (item[4] - item[5]))
    
    walks_score = item[6]
    
    game_score = (50 + outs_score + innings_after_four_score + strikeouts_score - hits_score - earned_runs_score - runs_score - walks_score)
    
    game_score_list.append(game_score)


pitchers_only_df['Game Score'] = game_score_list
pitchers_only_df.sort_values('Game Score', ascending = False).head(50)


/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/2628201052.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitchers_only_df['Game Score'] = game_score_list


,Name,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str,Ctct,StS,StL,GB,FB,LD,Unk,GSc,IR,IS,WPA,aLI,cWPA,acLI,RE24,Date,Game Score
117,"Clayton Kershaw, W (5-3)",Los Angeles Dodgers,8.0,1,0,0,2,11,0,3.36,27,106,75,36,18,21,9,4,1,0,89.0,NaN,NaN,0.486,0.98,0.40%,1.36,3.5,6/6/2015,89.0
114,"Mike Bolsinger, W (3-0)",Los Angeles Dodgers,8.0,1,0,0,0,8,0,0.71,24,92,60,27,10,23,10,6,3,0,88.0,NaN,NaN,0.482,0.86,0.42%,1.25,3.5,5/23/2015,88.0
37,"Barry Zito, W (3-0)",San Francisco Giants,8.0,3,0,0,3,10,0,1.32,28,114,72,40,14,18,11,4,0,0,83.0,NaN,NaN,0.536,1.17,0.35%,1.28,3.8,4/24/2010,83.0
50,Tim Lincecum,San Francisco Giants,8.0,5,0,0,1,11,0,5.93,30,114,75,37,13,25,6,12,3,0,82.0,NaN,NaN,0.462,1.17,0.44%,1.87,3.4,7/14/2012,82.0
277,"Mike Minor, W (3-2)",Texas Rangers,7.0,3,1,1,1,13,1,2.88,26,111,76,35,19,22,6,5,1,0,79.0,NaN,NaN,0.038,0.24,0.02%,0.25,2.6,4/27/2019,79.0
39,Chad Billingsley,Los Angeles Dodgers,7.0,4,1,1,1,13,0,3.61,26,121,81,41,15,25,6,6,2,0,77.0,NaN,NaN,0.361,1.22,0.00%,0.00,2.6,9/26/2010,77.0
291,"Wily Peralta, W (2-1)",Detroit Tigers,7.0,3,0,0,0,6,0,2.14,24,91,61,35,10,16,14,4,3,0,77.0,NaN,NaN,0.325,0.76,0.05%,0.18,3.6,7/5/2021,77.0
71,"Lance Lynn, W (7-1)",St. Louis Cardinals,6.0,2,0,0,1,9,0,2.95,20,77,53,24,8,21,4,6,1,0,76.0,NaN,NaN,0.226,0.68,0.17%,0.83,2.6,5/24/2013,76.0
186,"Carlos Martinez, W (4-4)",St. Louis Cardinals,8.0,4,1,1,3,9,0,3.08,30,106,70,33,12,25,8,10,6,0,76.0,NaN,NaN,0.399,1.28,0.26%,1.40,3.1,5/31/2017,76.0
100,"Hyun Jin Ryu, W (13-5)",Los Angeles Dodgers,7.0,2,0,0,1,4,0,3.21,25,100,58,31,7,20,12,7,4,0,76.0,NaN,NaN,0.291,0.70,0.23%,0.92,3.1,8/7/2014,76.0


In [26]:
pitching_record_df = pitchers_only_df[['Name','Date']].copy()

pitching_record_df['Decision'] = (np.where(pitching_record_df['Name'].str.contains(', '),
                  pitching_record_df['Name'].str.split(', ').str[1],
                  'ND'))

pitching_record_df['New_Name'] = (np.where(pitching_record_df['Name'].str.contains(', '),
                  pitching_record_df['Name'].str.split(', ').str[0],
                  pitching_record_df['Name']))

pitching_record_df['Record'] = (np.where(pitching_record_df['Decision'].str.contains(' '),
                  pitching_record_df['Decision'].str.split(' ').str[1],
                  ''))

pitching_record_df['Decision'] = (np.where(pitching_record_df['Decision'].str.contains(' '),
                  pitching_record_df['Decision'].str.split(' ').str[0],
                  pitching_record_df['Decision']))

pitching_record_df

,Name,Date,Decision,New_Name,Record
0,Barry Zito,7/27/2007,ND,Barry Zito,
1,Kevin Correia,7/27/2007,ND,Kevin Correia,
2,"Vinnie Chulk, W (4-2)",7/27/2007,W,Vinnie Chulk,(4-2)
3,Jonathan Sánchez,7/27/2007,ND,Jonathan Sánchez,
4,Randy Messenger,7/27/2007,ND,Randy Messenger,
...,...,...,...,...,...
321,Joely Rodríguez,7/15/2023,ND,Joely Rodríguez,
322,Joe Jacques,7/15/2023,ND,Joe Jacques,
323,Brandon Walter,7/15/2023,ND,Brandon Walter,
325,"Alex Cobb, L (7-6)",9/3/2023,L,Alex Cobb,(7-6)


In [27]:
pitchers_only_df = pd.merge(pitchers_only_df, pitching_record_df, on=['Name','Date'])

name_column = pitchers_only_df.pop('Name') 
pitchers_only_df.insert(0, 'Name', name_column) 

refinded_pitchers_df = pitchers_only_df[['Name', 'Team', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'ERA', 'BF',
       'Pit', 'Str','Date', 'Game Score','Decision']]


In [28]:
refinded_pitchers_df.reset_index(drop=True)

refinded_pitchers_df['Opponent'] = ''

i = 0   
    
for index, row in refinded_pitchers_df.iterrows():
    temp_df = games_attended_df.loc[games_attended_df['Date'] == row['Date']]
    if row['Team'] == temp_df['Home'].values[0]:
        refinded_pitchers_df['Opponent'][i] = temp_df['Visitor'].values[0]
    else:
        refinded_pitchers_df['Opponent'][i] = temp_df['Home'].values[0]
        
    i = i + 1


refinded_pitchers_df['Date']= pd.to_datetime(refinded_pitchers_df['Date'])

refinded_pitchers_df.sort_values('Date', ascending = True)

/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/2700434500.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refinded_pitchers_df['Opponent'] = ''
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/2700434500.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refinded_pitchers_df['Opponent'][i] = temp_df['Visitor'].values[0]
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/2700434500.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

,Name,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str,Date,Game Score,Decision,Opponent
0,Barry Zito,San Francisco Giants,4.0,8,6,6,4,3,1,5.28,23,87,50,2007-07-27,21.0,ND,Florida Marlins
1,Kevin Correia,San Francisco Giants,1.1,2,1,1,1,2,1,4.21,7,27,15,2007-07-27,47.0,ND,Florida Marlins
2,"Vinnie Chulk, W (4-2)",San Francisco Giants,0.2,0,0,0,1,0,0,3.35,3,10,5,2007-07-27,51.0,W,Florida Marlins
3,Jonathan Sánchez,San Francisco Giants,1.2,2,1,1,1,1,0,4.45,8,32,19,2007-07-27,47.0,ND,Florida Marlins
4,Randy Messenger,San Francisco Giants,1.1,3,2,2,0,1,0,2.15,7,22,15,2007-07-27,41.0,ND,Florida Marlins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,Scott Barlow,San Diego Padres,1.0,0,0,0,1,1,0,4.95,4,20,11,2023-09-03,53.0,ND,San Francisco Giants
535,"Alex Cobb, L (7-6)",San Francisco Giants,3.0,6,4,4,0,3,2,3.74,15,58,43,2023-09-03,34.0,L,San Diego Padres
267,"Seth Lugo, W (6-6)",San Diego Padres,6.0,3,0,0,1,4,0,3.49,22,89,55,2023-09-03,69.0,W,San Francisco Giants
270,Josh Hader,San Diego Padres,1.0,0,0,0,1,3,0,1.13,4,22,12,2023-09-03,55.0,ND,San Francisco Giants


In [29]:
refinded_pitchers_df.sort_values('Name', ascending = True)

,Name,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str,Date,Game Score,Decision,Opponent
75,"A.J. Burnett, W (6-8)",Philadelphia Phillies,7.2,5,2,2,4,6,1,3.83,31,108,66,2014-07-11,63.0,W,Washington Nationals
196,A.J. Minter,Atlanta Braves,1.0,1,0,0,0,1,0,3.30,3,17,11,2018-06-22,52.0,ND,Baltimore Orioles
528,"Aaron Bummer, W (2-1)",Chicago White Sox,1.1,0,0,0,1,3,0,6.65,5,15,11,2023-06-17,56.0,W,Seattle Mariners
176,Aaron Loup,Toronto Blue Jays,1.0,1,0,0,0,1,0,3.88,3,15,10,2017-09-23,52.0,ND,New York Yankees
523,"Aaron Loup, BS (1), L (0-1)",Los Angeles Angels,0.1,2,2,2,0,1,0,54.00,3,15,11,2023-03-30,40.0,BS,Oakland Athletics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,"Zach McAllister, L (1-1)",Cleveland Indians,0.2,1,1,1,0,1,1,2.27,3,12,9,2017-08-05,47.0,L,New York Yankees
502,Zach Plesac,Cleveland Indians,5.0,6,2,2,1,6,0,3.81,21,74,49,2019-09-27,52.0,ND,Washington Nationals
478,Zack Britton,Baltimore Orioles,0.1,5,4,4,0,0,0,7.71,6,16,13,2018-06-22,25.0,ND,Atlanta Braves
468,"Zack Greinke, L (0-1)",Arizona Diamondbacks,5.0,9,5,5,0,5,1,5.06,24,86,60,2018-04-07,34.0,L,St. Louis Cardinals


In [30]:
refinded_pitchers_df['Temp_Name'] = ' '

/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/3641003576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refinded_pitchers_df['Temp_Name'] = ' '


In [31]:
cleaned_name_list = pitching_record_df['New_Name'].to_list()

In [32]:
refinded_pitchers_df['Temp_Name'] = cleaned_name_list

/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_12528/4285783682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refinded_pitchers_df['Temp_Name'] = cleaned_name_list


In [33]:
refinded_pitchers_df = refinded_pitchers_df.drop('Name', axis=1)

refinded_pitchers_df = refinded_pitchers_df.rename(columns={"Temp_Name": "Name"})

temp_list = refinded_pitchers_df.pop('Name') 

refinded_pitchers_df.insert(0, 'Name', temp_list) 

refinded_pitchers_df

,Name,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str,Date,Game Score,Decision,Opponent
0,Barry Zito,San Francisco Giants,4.0,8,6,6,4,3,1,5.28,23,87,50,2007-07-27,21.0,ND,Florida Marlins
1,Kevin Correia,San Francisco Giants,1.1,2,1,1,1,2,1,4.21,7,27,15,2007-07-27,47.0,ND,Florida Marlins
2,Vinnie Chulk,San Francisco Giants,0.2,0,0,0,1,0,0,3.35,3,10,5,2007-07-27,51.0,W,Florida Marlins
3,Jonathan Sánchez,San Francisco Giants,1.2,2,1,1,1,1,0,4.45,8,32,19,2007-07-27,47.0,ND,Florida Marlins
4,Randy Messenger,San Francisco Giants,1.1,3,2,2,0,1,0,2.15,7,22,15,2007-07-27,41.0,ND,Florida Marlins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Joely Rodríguez,Boston Red Sox,1.0,0,0,0,0,2,0,12.00,3,10,8,2023-07-15,55.0,ND,Chicago Cubs
533,Joe Jacques,Boston Red Sox,1.0,0,0,0,0,1,0,4.63,3,14,8,2023-07-15,54.0,ND,Chicago Cubs
534,Brandon Walter,Boston Red Sox,2.0,6,2,2,0,3,0,3.29,11,34,24,2023-07-15,39.0,ND,Chicago Cubs
535,Alex Cobb,San Francisco Giants,3.0,6,4,4,0,3,2,3.74,15,58,43,2023-09-03,34.0,L,San Diego Padres


In [34]:
# refinded_pitchers_df.sort_values('Pit',ascending = False).head(25)


temp_refinded_pitchers_df = refinded_pitchers_df

temp_refinded_pitchers_df['Games'] = 1
temp_refinded_pitchers_df

,Name,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str,Date,Game Score,Decision,Opponent,Games
0,Barry Zito,San Francisco Giants,4.0,8,6,6,4,3,1,5.28,23,87,50,2007-07-27,21.0,ND,Florida Marlins,1
1,Kevin Correia,San Francisco Giants,1.1,2,1,1,1,2,1,4.21,7,27,15,2007-07-27,47.0,ND,Florida Marlins,1
2,Vinnie Chulk,San Francisco Giants,0.2,0,0,0,1,0,0,3.35,3,10,5,2007-07-27,51.0,W,Florida Marlins,1
3,Jonathan Sánchez,San Francisco Giants,1.2,2,1,1,1,1,0,4.45,8,32,19,2007-07-27,47.0,ND,Florida Marlins,1
4,Randy Messenger,San Francisco Giants,1.1,3,2,2,0,1,0,2.15,7,22,15,2007-07-27,41.0,ND,Florida Marlins,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Joely Rodríguez,Boston Red Sox,1.0,0,0,0,0,2,0,12.00,3,10,8,2023-07-15,55.0,ND,Chicago Cubs,1
533,Joe Jacques,Boston Red Sox,1.0,0,0,0,0,1,0,4.63,3,14,8,2023-07-15,54.0,ND,Chicago Cubs,1
534,Brandon Walter,Boston Red Sox,2.0,6,2,2,0,3,0,3.29,11,34,24,2023-07-15,39.0,ND,Chicago Cubs,1
535,Alex Cobb,San Francisco Giants,3.0,6,4,4,0,3,2,3.74,15,58,43,2023-09-03,34.0,L,San Diego Padres,1


In [35]:
def sum_innings(sum_list):
    whole_innings_list = []
    partial_innings_list = []
    for item in sum_list:
        split_number = math.modf(item)
        whole_innings_list.append(split_number[1])
        partial_innings_list.append(round(split_number[0],1) * 10)
    whole_innings = sum(whole_innings_list)
    partial_innings_decimal = sum(partial_innings_list)/3
    split_partial_innings = math.modf(partial_innings_decimal)
    
    if round(split_partial_innings[0],1) == 0.3:
        partial_inning = 0.1
    elif round(split_partial_innings[0],1) == 0.7:
        partial_inning = 0.2
    else:
        partial_inning = 0
    
    partial_innings = split_partial_innings[1] + partial_inning
    
    return whole_innings + partial_innings

In [36]:
pitch_col_list = temp_refinded_pitchers_df.columns.tolist()
pitch_col_list.remove('Name')
pitch_col_list.remove('Team')
pitch_col_list.remove('ERA')
pitch_col_list.remove('Decision')
pitch_col_list.remove('Opponent')
pitch_col_list.remove('Date')
pitch_col_list.remove('Game Score')

total_sum_list = []

for col in pitch_col_list: 
    sum_list = []
    sum_list = temp_refinded_pitchers_df.groupby('Name')[col].sum()
    total_sum_list.append(sum_list)
    
pitchers_list = temp_refinded_pitchers_df["Name"].values.tolist()
teams_list = temp_refinded_pitchers_df["Team"].values.tolist()
new_teams_list = []
        
for i in range(len(pitchers_list)):
    temp_list = []
    temp_list.append(pitchers_list[i])
    temp_list.append(teams_list[i])
    new_teams_list.append(temp_list)


    
new_pitcher_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
new_pitcher_teams_df = new_pitcher_teams_df.drop_duplicates()
new_pitcher_teams_df = new_pitcher_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
new_pitcher_teams_df = new_pitcher_teams_df.set_index('Name')    
    
pitcher_stat_sum_df = pd.DataFrame(total_sum_list)
pitcher_stat_sum_df = pitcher_stat_sum_df.transpose()


temp_innings_list = temp_refinded_pitchers_df.groupby('Name')['IP'].apply(list)
innings_summed_list = []
for item in temp_innings_list:
    innings_summed_list.append((sum_innings(item)))

pitcher_stat_sum_df['IP'] = innings_summed_list

pitcher_stat_sum_df = pitcher_stat_sum_df.join(new_pitcher_teams_df)
pitcher_stat_sum_df.sort_values('Games', ascending = False).head(50)

,IP,H,R,ER,BB,SO,HR,BF,Pit,Str,Games,Team
Name,,,,,,,,,,,,
Trevor Rosenthal,5.0,1.0,0.0,0.0,3.0,8.0,0.0,19.0,97.0,57.0,5.0,St. Louis Cardinals
Matt Bowman,3.2,5.0,1.0,1.0,3.0,3.0,0.0,17.0,64.0,41.0,5.0,St. Louis Cardinals
Adam Wainwright,34.0,30.0,14.0,11.0,11.0,26.0,0.0,140.0,515.0,337.0,5.0,St. Louis Cardinals
David Robertson,6.1,6.0,3.0,3.0,0.0,9.0,3.0,25.0,92.0,61.0,5.0,New York Yankees/Chicago White Sox
Jonathan Broxton,3.1,5.0,1.0,1.0,1.0,5.0,1.0,16.0,70.0,46.0,4.0,Los Angeles Dodgers/Cincinnati Reds/St. Louis ...
Brett Cecil,3.2,3.0,2.0,2.0,2.0,2.0,0.0,16.0,48.0,29.0,4.0,St. Louis Cardinals/Toronto Blue Jays
Michael Wacha,21.1,22.0,14.0,14.0,12.0,21.0,3.0,96.0,345.0,214.0,4.0,St. Louis Cardinals
Randy Choate,2.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,25.0,16.0,4.0,St. Louis Cardinals
Edinson Volquez,9.0,11.0,7.0,6.0,10.0,7.0,1.0,48.0,188.0,105.0,3.0,San Diego Padres/Los Angeles Dodgers/Kansas Ci...


In [37]:
innings_pitched_list = pitcher_stat_sum_df['IP'].to_list()
earned_runs_list = pitcher_stat_sum_df['ER'].to_list()
walks_list = pitcher_stat_sum_df['BB'].to_list()
hits_list = pitcher_stat_sum_df['H'].to_list()
whip_list = []
ERA_list = []

for i in range(len(innings_pitched_list)):
    if innings_pitched_list[i] > 0:
        ERA = ((9 * earned_runs_list[i]) / innings_pitched_list[i])
        ERA_list.append(ERA)
        WHIP = ((walks_list[i] + hits_list[i]) / innings_pitched_list[i])
        whip_list.append(WHIP)
    else:
        ERA_list.append(float('inf'))
        whip_list.append(float('inf'))
    

pitcher_stat_sum_df['ERA'] = ERA_list
pitcher_stat_sum_df['WHIP'] = whip_list
pitcher_stat_sum_df.loc[:, 'ERA'] = pitcher_stat_sum_df['ERA'].map('{:.2f}'.format)
pitcher_stat_sum_df.loc[:, 'WHIP'] = pitcher_stat_sum_df['WHIP'].map('{:.2f}'.format)
pitcher_stat_sum_df.loc[:, 'H'] = pitcher_stat_sum_df['H'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'R'] = pitcher_stat_sum_df['R'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'ER'] = pitcher_stat_sum_df['ER'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'BB'] = pitcher_stat_sum_df['BB'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'SO'] = pitcher_stat_sum_df['SO'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'HR'] = pitcher_stat_sum_df['HR'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'BF'] = pitcher_stat_sum_df['BF'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'Pit'] = pitcher_stat_sum_df['Pit'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'Str'] = pitcher_stat_sum_df['Str'].map('{:.0f}'.format)
pitcher_stat_sum_df.loc[:, 'Games'] = pitcher_stat_sum_df['Games'].map('{:.0f}'.format)

pitcher_stat_sum_df.sort_values('ERA', ascending = False).head(50)

,IP,H,R,ER,BB,SO,HR,BF,Pit,Str,Games,Team,ERA,WHIP
Name,,,,,,,,,,,,,,
Jake Faria,0.0,0,3,3,3,0,0,3,16,4,1,Tampa Bay Rays,inf,inf
Clayton Richard,0.0,1,0,0,0,0,0,2,3,2,1,Chicago Cubs,inf,inf
Derek Holland,0.0,0,3,3,3,0,0,3,13,1,1,Texas Rangers,inf,inf
Brett Myers,0.0,3,1,1,0,0,0,3,10,7,1,Houston Astros,inf,inf
Dean Kiekhefer,0.0,2,1,1,1,0,0,3,8,3,1,St. Louis Cardinals,inf,inf
Kodi Whitley,0.0,0,3,3,4,0,0,4,26,10,1,St. Louis Cardinals,inf,inf
Sergio Santos,0.0,2,2,1,0,0,0,2,3,2,1,Toronto Blue Jays,inf,inf
George Sherrill,0.0,2,3,3,1,0,1,3,10,5,1,Los Angeles Dodgers,inf,inf
Jacob Rhame,0.1,2,1,1,0,0,0,3,7,5,1,New York Mets,90.00,20.00


In [38]:
pitch_col_list = temp_refinded_pitchers_df.columns.tolist()
pitch_col_list.remove('Name')
pitch_col_list.remove('Team')
pitch_col_list.remove('ERA')
pitch_col_list.remove('Decision')
pitch_col_list.remove('Opponent')
pitch_col_list.remove('Date')

total_average_list = []

for col in pitch_col_list: 
    average_list = []
    teams_list = []
    
    if col == "Games":
        average_list = temp_refinded_pitchers_df.groupby('Name')[col].sum()
        total_average_list.append(average_list)
        
    elif col == "Team":           
        players_list = temp_refinded_pitchers_df["Name"].values.tolist()
        teams_list = temp_refinded_pitchers_df["Team"].values.tolist()
        new_teams_list = []
        
        for i in range(len(players_list)):
            temp_list = []
            temp_list.append(players_list[i])
            temp_list.append(teams_list[i])
            new_teams_list.append(temp_list)
        
        new_teams_df = pd.DataFrame(new_teams_list, columns = ("Name", "Team"))
        new_teams_df = new_teams_df.drop_duplicates()

        new_teams_df = new_teams_df.groupby('Name')['Team'].apply('/'.join).reset_index()
        new_teams_df = new_teams_df.set_index('Name')

    else:
        average_list = temp_refinded_pitchers_df.groupby('Name')[col].mean()
        total_average_list.append(average_list)
    
temp_pitcher_stat_average_df = pd.DataFrame(total_average_list)
temp_pitcher_stat_average_df = temp_pitcher_stat_average_df.transpose()
pitcher_stat_average_df = pd.concat([temp_pitcher_stat_average_df, new_teams_df], axis=1, join="inner")


pitcher_stat_average_df = pitcher_stat_average_df.round(2)

first_column2 = pitcher_stat_average_df.pop('Games')
second_column2 = pitcher_stat_average_df.pop('Team')

pitcher_stat_average_df.insert(0, 'Team', second_column2)
pitcher_stat_average_df.insert(0, 'Games', first_column2)

pitcher_stat_average_df

,Games,Team,IP,H,R,ER,BB,SO,HR,BF,Pit,Str,Game Score
Name,,,,,,,,,,,,,
A.J. Burnett,1.0,Philadelphia Phillies,7.20,5.0,2.0,2.0,4.0,6.0,1.0,31.0,108.0,66.0,63.0
A.J. Minter,1.0,Atlanta Braves,1.00,1.0,0.0,0.0,0.0,1.0,0.0,3.0,17.0,11.0,52.0
Aaron Bummer,1.0,Chicago White Sox,1.10,0.0,0.0,0.0,1.0,3.0,0.0,5.0,15.0,11.0,56.0
Aaron Loup,2.0,Los Angeles Angels/Toronto Blue Jays,0.55,1.5,1.0,1.0,0.0,1.0,0.0,3.0,15.0,10.5,46.0
Adam Cimber,1.0,Cleveland Indians,0.10,1.0,1.0,1.0,0.0,1.0,1.0,2.0,7.0,7.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zach McAllister,1.0,Cleveland Indians,0.20,1.0,1.0,1.0,0.0,1.0,1.0,3.0,12.0,9.0,47.0
Zach Plesac,1.0,Cleveland Indians,5.00,6.0,2.0,2.0,1.0,6.0,0.0,21.0,74.0,49.0,52.0
Zack Britton,1.0,Baltimore Orioles,0.10,5.0,4.0,4.0,0.0,0.0,0.0,6.0,16.0,13.0,25.0


In [39]:
pitcher_stat_sum_df.sort_values('IP')


,IP,H,R,ER,BB,SO,HR,BF,Pit,Str,Games,Team,ERA,WHIP
Name,,,,,,,,,,,,,,
Clayton Richard,0.0,1,0,0,0,0,0,2,3,2,1,Chicago Cubs,inf,inf
Derek Holland,0.0,0,3,3,3,0,0,3,13,1,1,Texas Rangers,inf,inf
Dean Kiekhefer,0.0,2,1,1,1,0,0,3,8,3,1,St. Louis Cardinals,inf,inf
Jake Faria,0.0,0,3,3,3,0,0,3,16,4,1,Tampa Bay Rays,inf,inf
Sergio Santos,0.0,2,2,1,0,0,0,2,3,2,1,Toronto Blue Jays,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Barry Zito,15.2,18,12,12,9,16,2,71,286,172,3,San Francisco Giants,7.11,1.78
Lance Lynn,16.2,13,4,4,5,14,0,65,261,173,3,St. Louis Cardinals,2.22,1.11
Hyun Jin Ryu,19.1,13,2,2,3,14,0,74,286,181,3,Los Angeles Dodgers,0.94,0.84


In [40]:
# Condense results to show total record from games I have seen

pitching_record_df

,Name,Date,Decision,New_Name,Record
0,Barry Zito,7/27/2007,ND,Barry Zito,
1,Kevin Correia,7/27/2007,ND,Kevin Correia,
2,"Vinnie Chulk, W (4-2)",7/27/2007,W,Vinnie Chulk,(4-2)
3,Jonathan Sánchez,7/27/2007,ND,Jonathan Sánchez,
4,Randy Messenger,7/27/2007,ND,Randy Messenger,
...,...,...,...,...,...
321,Joely Rodríguez,7/15/2023,ND,Joely Rodríguez,
322,Joe Jacques,7/15/2023,ND,Joe Jacques,
323,Brandon Walter,7/15/2023,ND,Brandon Walter,
325,"Alex Cobb, L (7-6)",9/3/2023,L,Alex Cobb,(7-6)


In [41]:
# Make sure there are no incorrect entries like Billy Hamilton in the HoF DF

allstars_seen_df.sort_values(by=['Appearances'], ascending=False)

,Player,Appearances
10,,6117
212,Tony Gwynn,15
194,Barry Bonds,14
239,Derek Jeter,14
1,Miguel Cabrera,12
...,...,...
202,Ryan Franklin,1
113,Jim Johnson,1
204,Chone Figgins,1
112,Erick Aybar,1


In [42]:
# Make sure there are no incorrect entries like Billy Hamilton in the HoF DF

mvp_seen_df.drop_duplicates(subset=['Player'])

,index,Player,Games,Team,AB,R,H,RBI,BB,SO,PA,PO,A,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Year,League
0,7,Aaron Judge,2.0,New York Yankees,4.00,0.00,1.00,0.00,0.00,2.00,4.00,3.50,0.00,0.00,0.50,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2022,AL
1,36,Albert Pujols,7.0,St. Louis Cardinals/Los Angeles Angels of Anaheim,4.00,0.57,0.71,0.43,0.43,0.57,4.43,7.71,0.71,0.14,0.43,0.0,0.00,0.29,0.0,0.0,0.0,0.0,0.0,2009,NL
4,58,Andrew McCutchen,2.0,Pittsburgh Pirates,4.50,0.00,1.50,1.50,0.00,1.00,4.50,2.00,0.00,0.00,0.50,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2013,NL
5,89,Barry Bonds,2.0,San Francisco Giants,2.00,1.00,0.50,0.50,2.50,0.50,4.50,2.00,0.00,0.50,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2004,NL
12,161,Bryce Harper,2.0,Washington Nationals,3.00,1.00,0.50,0.50,0.50,0.50,4.00,2.00,0.50,0.50,0.00,0.0,0.00,0.00,0.0,0.0,0.5,0.0,0.0,2021,NL
14,165,Buster Posey,5.0,San Francisco Giants,3.20,0.60,1.20,0.80,0.80,0.60,4.00,5.20,0.20,0.20,0.00,0.0,0.00,0.00,0.0,0.2,0.0,0.0,0.0,2012,NL
15,223,Clayton Kershaw,1.0,Los Angeles Dodgers,2.00,0.00,0.00,0.00,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2014,NL
16,229,Cody Bellinger,3.0,Los Angeles Dodgers/Chicago Cubs,3.67,0.67,1.00,1.33,0.67,0.67,4.33,2.00,0.00,0.33,0.33,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2019,NL
17,313,Dustin Pedroia,1.0,Boston Red Sox,4.00,2.00,2.00,1.00,0.00,0.00,4.00,1.00,1.00,0.00,2.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2008,AL
18,367,Freddie Freeman,3.0,Atlanta Braves,4.67,0.67,0.67,0.67,0.67,1.00,5.33,9.00,0.67,0.00,0.33,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,2020,NL


In [43]:
# Make sure there are no incorrect entries like Billy Hamilton 

hof_seen_df

,Name,Games,Team,AB,R,H,RBI,BB,SO,PA,PO,A,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Life,Voted By,Inducted As,Votes,% of Ballots
0,Billy Hamilton,2.0,Cincinnati Reds,4.0,1.5,1.0,0.0,0.5,1.0,4.5,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,1866-1940,Veterans,Player,NaN,NaN
1,David Ortiz,1.0,Boston Red Sox,3.0,0.0,0.0,0.0,0.0,2.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1975-Living,BBWAA,Player,307.0,77.9%
2,Derek Jeter,1.0,New York Yankees,5.0,0.0,1.0,1.0,0.0,1.0,5.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1974-Living,BBWAA,Player,396.0,99.7%
3,Joe Mauer,2.0,Minnesota Twins,4.5,0.5,1.0,0.0,0.0,0.5,4.5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1983-Living,BBWAA,Player,293.0,76.1%
4,Tony Gwynn,2.0,San Diego Padres/Philadelphia Phillies,2.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1960-2014,BBWAA,Player,532.0,97.6%


In [44]:
# Clean up columns nobody cares about

player_stat_sum_df.iloc[350:400]

,AB,R,H,RBI,BB,SO,PA,WPA,PO,A,Games,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,BA,OBP,SLG,OPS,Team,Pos
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Evan Scribner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,Oakland Athletics,P
Everth Cabrera,6.0,0.0,1.0,0.0,1.0,3.0,7.0,-0.054,2.0,6.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.167,0.286,0.167,0.452,San Diego Padres,SS
Ezequiel Carrera,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.041,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,Toronto Blue Jays,PH
Felipe Lopez,8.0,0.0,3.0,2.0,0.0,3.0,9.0,0.045,3.0,3.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.375,0.333,0.375,0.708,St. Louis Cardinals,2B/RF/LF
Felix Doubront,3.0,0.0,0.0,0.0,0.0,1.0,4.0,-0.012,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,Boston Red Sox,P
Fernando Abad,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,Houston Astros/Oakland Athletics,P
Fernando Rodriguez Jr.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,Houston Astros,P
Fernando Salas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,Arizona Diamondbacks/Los Angeles Angels of Ana...,P
Fernando Tatis Jr.,4.0,1.0,0.0,0.0,0.0,0.0,4.0,-0.066,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,San Diego Padres,RF


In [45]:
# Clean up columns nobody cares about

player_stat_average_df.columns

Index(['index', 'Player', 'Games', 'Team', 'AB', 'R', 'H', 'RBI', 'BB', 'SO',
       'PA', 'PO', 'A', 'HR', '2B', 'SF', 'IW', 'GDP', 'SH', 'SB', 'HBP', '3B',
       'CS'],
      dtype='object')

In [46]:
# Clean up columns nobody cares about

new_all_player_stats_df.columns

refinded_players_stats_df = new_all_player_stats_df[['Name', 'Team', 'Date','PA','AB','R','H','2B','3B','HR','RBI',
                'BB','IW','SO','OBP','Pit','Str','WPA','RE24','PO','A','SF','SH','GDP','HBP','SB','CS','Pos']]

refinded_players_stats_df.sort_values(by=['PA'], ascending=True).head(533)

,Name,Team,Date,PA,AB,R,H,2B,3B,HR,RBI,BB,IW,SO,OBP,Pit,Str,WPA,RE24,PO,A,SF,SH,GDP,HBP,SB,CS,Pos
926,Randy Messenger,San Francisco Giants,7/27/2007,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,0,0,0,0,0,P
873,Ryan Tepera,Los Angeles Angels,3/30/2023,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,0,0,0,0,0,P
879,Romy Gonzalez,Chicago White Sox,6/17/2023,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.208,0.0,0.0,0.000,0.0,0.0,0.0,0,0,0,0,0,0,PR-DH-DH
887,,Chicago White Sox,6/17/2023,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,0,0,0,0,0,
888,Lucas Giolito,Chicago White Sox,6/17/2023,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,0,0,0,0,0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Edward Mujica,St. Louis Cardinals,10/14/2012,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0,0,0,0,0,0,P
1134,Chris Capuano,Los Angeles Dodgers,5/24/2013,1.0,1.0,0.0,0.0,0,0,0,0.0,0.0,0,1.0,0.000,3.0,3.0,-0.034,-0.3,0.0,0.0,0,0,0,0,0,0,P
1052,Adam LaRoche,Arizona Diamondbacks,9/26/2010,1.0,1.0,0.0,0.0,0,0,0,0.0,0.0,0,1.0,0.321,8.0,6.0,-0.036,-0.2,0.0,0.0,0,0,0,0,0,0,PH
1397,Willson Contreras,Chicago Cubs,7/29/2016,1.0,1.0,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.360,2.0,2.0,0.000,-0.4,2.0,0.0,0,0,0,0,0,0,PH-C


In [47]:
refinded_players_stats_df = refinded_players_stats_df[refinded_players_stats_df.PA != 0.0]
refinded_players_stats_df.sort_values(by=['PA'], ascending=True)

,Name,Team,Date,PA,AB,R,H,2B,3B,HR,RBI,BB,IW,SO,OBP,Pit,Str,WPA,RE24,PO,A,SF,SH,GDP,HBP,SB,CS,Pos
915,Austin Slater,San Francisco Giants,9/3/2023,1.0,1.0,0.0,0.0,0,0,0,0.0,0.0,0,1.0,0.345,7.0,4.0,-0.004,-0.2,0.0,0.0,0,0,0,0,0,0,PH
700,Chris Owings,Arizona Diamondbacks,4/8/2018,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0,0.0,0.483,4.0,2.0,0.069,0.4,0.0,0.0,0,0,0,0,0,0,PH
1333,Daniel Descalso,Colorado Rockies,7/18/2016,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0,0.0,0.440,1.0,1.0,0.079,0.9,0.0,0.0,0,0,0,0,0,0,PH
1042,Andres Torres,San Francisco Giants,4/24/2010,1.0,1.0,1.0,1.0,0,0,0,1.0,0.0,0,0.0,0.269,4.0,3.0,0.153,0.8,0.0,0.0,0,0,0,0,0,0,PH-CF
1051,Joe Saunders,Arizona Diamondbacks,9/26/2010,1.0,1.0,0.0,0.0,0,0,0,0.0,0.0,0,1.0,0.130,5.0,4.0,-0.018,-0.2,0.0,0.0,0,0,0,0,0,0,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,Manny Machado,Baltimore Orioles,6/22/2018,7.0,5.0,2.0,2.0,0,0,1,2.0,2.0,0,0.0,0.374,34.0,21.0,0.753,3.1,1.0,6.0,0,0,0,0,1,0,SS
714,Chris Davis,Baltimore Orioles,6/22/2018,7.0,4.0,1.0,1.0,0,0,1,2.0,2.0,1,1.0,0.233,28.0,14.0,0.151,0.8,18.0,1.0,1,0,0,0,0,0,1B
713,Jonathan Schoop,Baltimore Orioles,6/22/2018,7.0,7.0,1.0,2.0,1,0,0,2.0,0.0,0,0.0,0.257,18.0,13.0,0.193,0.7,1.0,6.0,0,0,1,0,0,0,2B
1622,Ender Inciarte,Atlanta Braves,6/22/2018,8.0,6.0,1.0,2.0,0,0,0,0.0,1.0,0,1.0,0.313,31.0,19.0,0.060,0.3,4.0,0.0,0,0,0,1,0,0,CF


In [48]:
# pitcher_stat_sum_df.to_csv('pitcher_stat_sum.csv')
# pitcher_stat_average_df.to_csv('pitcher_stat_average_df')
# refinded_pitchers_df.to_csv('refinded_pitchers_df')
# total_pitching_df.to_csv('total_pitching_df')
# games_breakdown_df.to_csv('games_breakdown_df')
# allstars_seen_df.to_csv('allstars_seen_df')
# mvp_seen_df.to_csv('mvp_seen_df')
# hof_seen_df.to_csv('hof_seen_df')
# player_records_df.to_csv('player_records_df')
# team_records_df.to_csv('team_records_df')
# home_away_record_df.to_csv('home_away_record_df')
# player_stat_average_df.player_stat_average_df
# player_stat_sum_df.player_stat_sum_df
# new_all_player_stats_df.new_all_player_stats_df

In [49]:
new_all_player_stats_df.sort_values(by=['RE24'], ascending=False).head(50)


,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,SLG,OPS,Pit,Str,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Games,HR,2B,SF,IW,GDP,SH,SB,HBP,3B,CS,Pos
385,Stephen Piscotty,St. Louis Cardinals,4.0,2.0,3.0,5.0,1.0,0.0,5.0,0.324,0.354,0.562,0.916,19.0,9.0,0.130,0.55,0.151,-0.021%,0.07%,0.46,4.8,2.0,0.0,"2·HR,3B,IW",8/23/2015,1,2,0,0,1,0,0,0,0,1,0,LF-RF
366,Buster Posey,San Francisco Giants,4.0,2.0,3.0,4.0,1.0,0.0,5.0,0.297,0.374,0.464,0.839,19.0,12.0,0.457,1.31,0.479,-0.023%,0.48%,2.29,4.7,4.0,0.0,"HR,SB",6/19/2015,1,1,0,0,0,0,0,1,0,0,0,C
1759,Yordan Alvarez,Houston Astros,3.0,3.0,3.0,5.0,1.0,0.0,4.0,0.308,0.378,0.559,0.937,9.0,5.0,0.460,1.10,0.460,0.000%,0.48%,1.92,4.4,1.0,0.0,"2·HR,IW",7/6/2021,1,2,0,0,1,0,0,0,0,0,0,LF
1455,Victor Martinez,Detroit Tigers,4.0,2.0,2.0,4.0,1.0,0.0,5.0,0.293,0.355,0.483,0.838,14.0,6.0,0.184,0.94,0.210,-0.026%,0.28%,2.37,4.0,0.0,0.0,HR,9/25/2016,1,1,0,0,0,0,0,0,0,0,0,DH
1345,Kirk Nieuwenhuis,Milwaukee Brewers,3.0,2.0,3.0,4.0,1.0,0.0,4.0,0.207,0.321,0.380,0.702,17.0,8.0,0.151,0.42,0.159,-0.008%,0.00%,0.01,3.9,2.0,0.0,"2·HR,CS",7/23/2016,1,2,0,0,0,0,0,0,0,0,1,CF
1369,Adam Eaton,Chicago White Sox,3.0,3.0,2.0,3.0,2.0,0.0,5.0,0.277,0.362,0.406,0.768,25.0,13.0,0.323,0.99,0.330,-0.007%,0.10%,0.51,3.6,1.0,0.0,HR,7/24/2016,1,1,0,0,0,0,0,0,0,0,0,RF
772,Rougned Odor,Texas Rangers,4.0,3.0,1.0,3.0,2.0,1.0,6.0,0.157,0.259,0.255,0.514,21.0,9.0,0.226,0.52,0.229,-0.003%,0.14%,0.54,3.6,0.0,0.0,HR,4/27/2019,1,1,0,0,0,0,0,0,0,0,0,2B
64,Jason Kubel,Minnesota Twins,4.0,2.0,3.0,3.0,1.0,0.0,5.0,0.315,0.383,0.570,0.952,21.0,11.0,0.215,0.48,0.216,-0.002,0.13%,0.48,3.5,3.0,0.0,"HR,2B",7/25/2009,1,1,1,0,0,0,0,0,0,0,0,LF
1394,Kris Bryant,Chicago Cubs,3.0,1.0,3.0,1.0,2.0,0.0,5.0,0.289,0.385,0.557,0.942,20.0,11.0,0.123,0.31,0.123,0.000%,0.06%,0.25,3.4,2.0,1.0,"2B,SB",7/29/2016,1,0,1,0,0,0,0,1,0,0,0,3B
548,Alex Gordon,Kansas City Royals,4.0,2.0,3.0,3.0,1.0,1.0,5.0,0.223,0.318,0.390,0.708,21.0,14.0,0.122,0.40,0.136,-0.014%,0%,0.00,3.4,1.0,0.0,"HR,2·2B",9/25/2016,1,1,2,0,0,0,0,0,0,0,0,LF


In [50]:
# all_player_stats_df['Name'] == 'Nick MartinezP'
# pd.set_option('display.max_columns', None)
# from sklearn.datasets import load_iris

all_player_stats_df.loc[all_player_stats_df['Name'] == "['Hanley RamirezSS']"]

,Name,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,SLG,OPS,Pit,Str,WPA,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Games
0,['Hanley RamirezSS'],Florida Marlins,5.0,1.0,3.0,4.0,0.0,0.0,6.0,0.339,0.391,0.565,0.956,21.0,12.0,0.174,1.01,0.245,-0.071,0.03%,0.26,1.7,4.0,2.0,"HR,2B,SF",7/27/2007,1
15,['Hanley RamirezSS'],Florida Marlins,4.0,1.0,2.0,0.0,0.0,0.0,4.0,0.341,0.392,0.567,0.959,20.0,10.0,0.054,0.83,0.086,-0.032,0.01%,0.18,0.4,1.0,8.0,"2B,SB",7/28/2007,1
29,['Hanley RamirezSS'],Florida Marlins,5.0,1.0,2.0,0.0,0.0,1.0,5.0,0.296,0.395,0.531,0.926,14.0,9.0,-0.041,2.11,0.122,-0.163,-0.00%,0.18,-0.2,1.0,1.0,0,9/5/2008,1
48,['Hanley RamirezSS'],Florida Marlins,4.0,0.0,1.0,0.0,0.0,0.0,4.0,0.296,0.394,0.529,0.923,19.0,10.0,-0.037,1.21,0.052,-0.089,-0.00%,0.17,-0.2,1.0,1.0,0,9/6/2008,1
78,['Hanley RamirezSS'],Florida Marlins,4.0,1.0,2.0,3.0,0.0,1.0,5.0,0.348,0.412,0.557,0.969,10.0,7.0,0.183,0.50,0.199,-0.016,0.12%,0.55,2.4,3.0,7.0,"2B,HBP",7/26/2009,1
1151,['Hanley RamirezSS'],Los Angeles Dodgers,4.0,2.0,2.0,2.0,0.0,0.0,4.0,0.346,0.393,0.634,1.027,13.0,11.0,0.020,0.85,0.081,-0.061%,0.00%,0.26,1.5,0.0,2.0,2·2B,8/30/2013,1


In [51]:
all_player_stats_df.columns

Index(['Name', 'Team', 'AB', 'R', 'H', 'RBI', 'BB', 'SO', 'PA', 'BA', 'OBP',
       'SLG', 'OPS', 'Pit', 'Str', 'WPA', 'aLI', 'WPA+', 'WPA-', 'cWPA',
       'acLI', 'RE24', 'PO', 'A', 'Details', 'Date', 'Games'],
      dtype='object')

In [52]:
player_stat_sum_df.loc["Hanley Ramirez"]

AB                                      31.0
R                                        6.0
H                                       15.0
RBI                                     11.0
BB                                       0.0
SO                                       3.0
PA                                      33.0
WPA                                    0.499
PO                                      10.0
A                                       21.0
Games                                    7.0
HR                                       1.0
2B                                       6.0
SF                                       1.0
IW                                       0.0
GDP                                      0.0
SH                                       0.0
SB                                       1.0
HBP                                      1.0
3B                                       0.0
CS                                       0.0
BA                                     0.484
OBP       

In [92]:
hof_df.loc[hof_df['Inducted As'] != "Player"]

,Unnamed: 0,Name,Unnamed: 2,Voted By,Inducted As,Votes,% of Ballots
2,2,Jim Leyland,1944-Living,Contemporary Baseball Era Non-Players,Manager,NaN,NaN
6,6,Bud Fowler,1858-1913,Early Baseball Era,Pioneer/Executive,NaN,NaN
10,10,Buck O'Neil,1911-2006,Early Baseball Era,Pioneer/Executive,NaN,NaN
14,14,Marvin Miller,1917-2012,Modern Baseball Era,Pioneer/Executive,NaN,NaN
32,32,John Schuerholz,1940-Living,Today's Game Era,Pioneer/Executive,NaN,NaN
...,...,...,...,...,...,...,...
333,333,Morgan Bulkeley,1837-1922,Centennial,Pioneer/Executive,NaN,NaN
334,334,Ban Johnson,1865-1931,Centennial,Pioneer/Executive,NaN,NaN
336,336,Connie Mack,1862-1956,Centennial,Manager,NaN,NaN
337,337,John McGraw,1873-1934,Centennial,Manager,NaN,NaN


In [66]:
MLB_players_df.loc[MLB_players_df['HOF'] != "NO"].sort_values(by=['HOF'], ascending=False).head(50)

,Name,Career Start,Career End,HOF
14582,Joe Mauer,2004,2018,2024
9832,Todd Helton,1997,2013,2024
1843,Adrián Beltré,1998,2018,2024
14925,Fred McGriff,1986,2004,2023
19597,Scott Rolen,1996,2012,2023
15507,Minnie Minoso,1949,1980,2022
11496,Jim Kaat,1959,1983,2022
16821,Tony Oliva,1962,1976,2022
16909,David Ortiz,1997,2016,2022
10225,Gil Hodges,1943,1963,2022


In [65]:
# MLB_players_df = MLB_players_df.drop(columns=['Unnamed: 0'])
MLB_players_df = MLB_players_df.drop_duplicates()
MLB_players_df

,Name,Career Start,Career End,HOF
0,David Aardsma,2004,2015,NO
1,Hank Aaron,1954,1976,1982
2,Tommie Aaron,1962,1971,NO
3,Don Aase,1977,1990,NO
4,Andy Abad,2001,2006,NO
...,...,...,...,...
24518,Frank Zupo,1957,1961,NO
24519,Paul Zuvella,1982,1991,NO
24520,George Zuverink,1951,1959,NO
24521,Dutch Zwilling,1910,1916,NO


In [68]:
MLB_players_df.loc[MLB_players_df['Name'] == "Mariano Rivera"]

,Name,Career Start,Career End,HOF
19334,Mariano Rivera,1995,2013,NO


In [79]:
hof_name_list = hof_df["Name"].to_list()

In [88]:
# MLB_players_df.loc[MLB_players_df['Name'] in hof_seen_name_list]

mask = MLB_players_df['Name'].isin(hof_name_list)
active_customers = MLB_players_df[mask]

# active_customers#.loc[active_customers['Name'] == "Mariano Rivera"]

active_customers = active_customers.drop_duplicates()
active_customers

,Name,Career Start,Career End,HOF
197,Grover Alexander,1911,1930,1938
279,Roberto Alomar,1988,2004,2011
292,Walter Alston,1936,1936,1983
402,Sparky Anderson,1959,1959,2000
445,Cap Anson,1876,1897,1939
...,...,...,...,...
24286,Early Wynn,1939,1963,1972
24315,Carl Yastrzemski,1961,1983,1989
24374,Cy Young,1890,1911,1937
24413,Ross Youngs,1917,1926,1972


In [78]:
new_all_player_stats_df[("Name","Date")]
refinded_pitchers_df

KeyError: ('Name', 'Date')

In [97]:
# new_all_player_stats_df.reset_index(inplace=True)

refinded_pitchers_df.reset_index(inplace=True)

temp_hof_players_seen_df = pd.merge(new_all_player_stats_df, hof_df, on="Name")
temp_hof_players_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)
temp_hof_players_seen_df.rename(columns = {'Unnamed: 2': 'Life'}, inplace = True)

# temp_hof_pitchers_seen_df = pd.merge(refinded_pitchers_df, hof_df, on="Name")
# temp_hof_pitchers_seen_df.drop('Unnamed: 0', axis = 1, inplace = True)
# temp_hof_pitchers_seen_df.rename(columns = {'Unnamed: 2': 'Life'}, inplace = True)

# temp_hof_pitchers_seen_df

,index,Name,Team,IP,H,R,ER,BB,SO,HR,ERA,BF,Pit,Str,Date,Game Score,Decision,Opponent,Games,Life,Voted By,Inducted As,Votes,% of Ballots


In [109]:
temp_hof_players_seen_list = temp_hof_players_seen_df['Name'].to_list()

mask = MLB_players_df['Name'].isin(temp_hof_players_seen_list)
active_customers = MLB_players_df[mask]

hof_players_seen_check_df = pd.merge(hof_df, MLB_players_df, on="Name")
hof_players_seen_check_df

# display(dataFrame.loc[(dataFrame['Salary']>=100000) & (dataFrame['Age']< 40) & (dataFrame['JOB'].str.startswith('D')),
#                     ['Name','JOB']])

,Unnamed: 0,Name,Unnamed: 2,Voted By,Inducted As,Votes,% of Ballots,Career Start,Career End,HOF
0,0,Adrián Beltré,1979-Living,BBWAA,Player,366.0,95.1%,1998,2018,2024
1,1,Todd Helton,1973-Living,BBWAA,Player,307.0,79.7%,1997,2013,2024
2,3,Joe Mauer,1983-Living,BBWAA,Player,293.0,76.1%,2004,2018,2024
3,4,Fred McGriff,1963-Living,Contemporary Era,Player,NaN,NaN,1986,2004,2023
4,5,Scott Rolen,1975-Living,BBWAA,Player,297.0,76.3%,1996,2012,2023
...,...,...,...,...,...,...,...,...,...,...
286,341,Ty Cobb,1886-1961,BBWAA,Player,222.0,98.2%,1905,1928,1936
287,342,Walter Johnson,1887-1946,BBWAA,Player,189.0,83.6%,1907,1927,1936
288,343,Christy Mathewson,1880-1925,BBWAA,Player,205.0,90.7%,1900,1916,1936
289,344,Babe Ruth,1895-1948,BBWAA,Player,215.0,95.1%,1914,1935,1936


In [112]:
hof_players_seen_check_df.loc[hof_players_seen_check_df["Name"] == "Billy Hamilton"]

,Unnamed: 0,Name,Unnamed: 2,Voted By,Inducted As,Votes,% of Ballots,Career Start,Career End,HOF
208,261,Billy Hamilton,1866-1940,Veterans,Player,NaN,NaN,1888,1901,1961
209,261,Billy Hamilton,1866-1940,Veterans,Player,NaN,NaN,2013,2023,NO
